Pada code block ini, kita mengimpor modul atau library yang dibutuhkan untuk menjalankan pipeline

In [1]:
import os
import sys
from typing import Text
 
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

Kemudian, pada dua code block di bawah ini, kita mendefinisikan beberapa variabel penting untuk pipeline, termasuk nama pipeline, lokasi data, modul transformasi dan pelatihan, serta jalur output untuk model yang disajikan dan metadata

In [2]:
PIPELINE_NAME = "red-wine-pipeline"
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = os.path.join("modules", "redwine_quality_transform.py")
TRAINER_MODULE_FILE = os.path.join("modules", "redwine_quality_train.py")
TUNER_MODULE = os.path.join("modules", "redwine_quality_tuner.py")

In [3]:
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

Di code block ini, kita mendefinisikan fungsi 'init_local_pipeline' yang bertanggung jawab untuk menginisialisasi pipeline TFX. Fungsi ini mengatur root pipeline, mengonfigurasi argumentasi Beam, dan mengembalikan objek pipeline TFX. Setelah itu, kita mengatur verbosity logging dan mengimpor fungsi 'init_components' dari modul yang telah dibuat. Selanjutnya, kita menginisialisasi komponen pipeline dengan memanggil 'init_components', menginisialisasi pipeline menggunakan 'init_local_pipeline', dan akhirnya menjalankan pipeline menggunakan BeamDagRunner.

In [4]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing",
        # 0 auto-detect based on on the number of CPUs available 
        # during execution time.
        "--direct_num_workers=0" 
    ]
    logging.info(f"Beam arguments: {beam_args}")
    
    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)
    
    from modules.components import init_components
    
    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        tuner_module = TUNER_MODULE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

Trial 30 Complete [00h 00m 19s]
val_mean_squared_error: 0.37576693296432495

Best val_mean_squared_error So Far: 0.32098597288131714
Total elapsed time: 00h 05m 53s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Int', 'config': {'name': 'units', 'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 32, 'sampling': None}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}], 'values': {'units': 128, 'learning_rate': 0.001, 'tuner/epochs': 10, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}}
INFO:absl:Best Hyperparameters are written to output\red-wine-pipeline\Tuner\best_hyperparameters\17\best_hyperparameters.txt.
INFO:absl:Tuner results are written to output\red-wine-pipeline\Tuner\tuner_results\17\tuner_results.json.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 17 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'best_h

Results summary
Results in output\red-wine-pipeline\Tuner\.system\executor_execution\17\.temp\17\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units: 128
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.32098597288131714
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.01
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.3508669137954712
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.01
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.35272929072380066
Trial summary
Hyperparameters:
units: 128
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.3567546606063843
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.357504665851593
Trial summary
Hyperparameters:
units: 256
lear

INFO:absl:node Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "red-wine-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20240716-210604.685488"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "red-wine-pipeline.Trainer"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channels {
        producer_node_query {
          id: "Transform"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "red-wine-pipeline"
            }
          }
        }
        conte

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fixed_acidity_xf (InputLayer)  [(None, 1)]          0           []                               
                                                                                                  
 volatile_acidity_xf (InputLaye  [(None, 1)]         0           []                               
 r)                                                                                               
                                                                                                  
 citric_acid_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 residual_sugar_xf (InputLayer)  [(None, 1)]         0           []                         

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: output\red-wine-pipeline\Trainer\model\18\Format-Serving\assets


INFO:tensorflow:Assets written to: output\red-wine-pipeline\Trainer\model\18\Format-Serving\assets
INFO:absl:Training complete. Model written to output\red-wine-pipeline\Trainer\model\18\Format-Serving. ModelRun written to output\red-wine-pipeline\Trainer\model_run\18
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 18 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "output\\red-wine-pipeline\\Trainer\\model_run\\18"
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "output\\red-wine-pipeline\\Trainer\\model\\18"
, artifact_type: name: "Model"
base_type: MODEL
)]}) for execution 18
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Trainer is finished.
INFO:absl:node Evaluator is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.evaluator.component.Evaluator"
    base_type: EVA

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to output\red-wine-pipeline\Evaluator\blessing\19.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 19 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'evaluation': [Artifact(artifact: uri: "output\\red-wine-pipeline\\Evaluator\\evaluation\\19"
, artifact_type: name: "ModelEvaluation"
)], 'blessing': [Artifact(artifact: uri: "output\\red-wine-pipeline\\Evaluator\\blessing\\19"
, artifact_type: name: "ModelBlessing"
)]}) for execution 19
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
  

Melalui output dari code block di atas, kita bisa melihat hasil dari setiap komponen pipeline TFX seperti hasil dari Tuner, Trainer, Evaluator, Pusher, dan lain - lain.